In [ ]:
import sklearn
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.model_selection import train_test_split
pd.options.display.max_columns = None
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rc('figure', figsize=[10,5])
import mlrose_hiive
from sklearn.metrics import f1_score
from functools import partial
from IPython.display import display
from sklearn.model_selection import train_test_split
pd.options.display.max_columns = None
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rc('figure', figsize=[10,5])
from sklearn import preprocessing
import pandas as pd
import numpy as np
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from collections import defaultdict, Counter
from sklearn.metrics import adjusted_mutual_info_score
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
import sys
from sklearn.metrics import accuracy_score, homogeneity_score
from sklearn import preprocessing

np.random.seed(88)
bank = pd.read_csv("banking_data.csv")
#bank.head()
min_max_scaler = preprocessing.MinMaxScaler()

bankX = bank.drop("Bankrupt", 1).copy().values
bankY = bank["Bankrupt"].copy().values
bankX = min_max_scaler.fit_transform(bankX)

In [ ]:
clusters = list(range(15,45,1))

In [ ]:
%%time
sse = {}
for cluster in list(range(1,150,1)):
    kmeans = KMeans(n_clusters=cluster, max_iter=1000, random_state=44, n_jobs=-2).fit(bankX)
    sse[cluster] = kmeans.inertia_
plt.figure()
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Values of K")
plt.ylabel("Squared Error")
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=2, n_init=50, max_iter=1000, random_state=44, n_jobs=-2).fit(bankX)
accuracy_score((1-kmeans.labels_), bankY)

In [ ]:
kmeans = KMeans(n_clusters=2, n_init=50, max_iter=1000, random_state=44, n_jobs=-2).fit(bankX)
adjusted_mutual_info_score((1-kmeans.labels_), bankY)

In [ ]:
%%time

from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

silh = {}
homog = {}
for cluster in range(15, 45, 1):
    kmeans = KMeans(n_clusters=cluster, n_init=50, max_iter=1000, random_state=44, 
                    n_jobs=-2).fit(bankX)
    label = kmeans.labels_
    sil_coeff = silhouette_score(bankX, label, metric='euclidean')
    homog_score = homogeneity_score(bankY, label)
    silh[cluster] = sil_coeff
    homog[cluster] = homog_score
    print("For n_clusters={}, The Silhouette Coefficient is {}".format(cluster, sil_coeff))
    print("For n_clusters={}, The homogeneity_score is {}".format(cluster, homog_score))
plt.figure()
plt.plot(list(silh.keys()), list(silh.values()))
plt.xlabel("Number of cluster")
plt.ylabel("Silhouette Coefficient")
plt.show()

In [ ]:
plt.figure()
plt.plot(list(silh.keys()), list(silh.values()), 'r', label="Silhouette")
plt.plot(list(homog.keys()), list(homog.values()), 'g', label="Homogeneity")
plt.legend()
plt.xlabel("Number of cluster")
plt.ylabel("Silhouette Coefficient")

In [ ]:
%%time
silh_EM = {}
homog_EM = {}
log_likelihood_EM = {}
for cluster in range(2, 15, 1):
    gmm = GaussianMixture(n_components=cluster, max_iter=100, random_state=44, 
                          n_init=5).fit(bankX)
    label = gmm.predict(bankX)
    sil_coeff = silhouette_score(bankX, label, metric='euclidean')
    homog_score = homogeneity_score(bankY, label)
    silh_EM[cluster] = sil_coeff
    homog_EM[cluster] = homog_score
    log_likelihood_EM[cluster] = gmm.score(bankX)
    print("For n_clusters={}, The Silhouette Coefficient is {}".format(cluster, sil_coeff))
    print("For n_clusters={}, The homogeneity_score is {}".format(cluster, homog_score))
    print("For n_clusters={}, The log_likelihood score is {}".format(cluster, log_likelihood_EM[cluster]))

    
plt.figure()
plt.plot(list(silh_EM.keys()), list(silh_EM.values()))
plt.xlabel("Number of cluster")
plt.ylabel("Silhouette Coefficient")
plt.show()

In [ ]:
plt.figure()
plt.plot(list(silh_EM.keys()), list(silh_EM.values()), 'r', label="Silhouette")
plt.plot(list(homog_EM.keys()), list(homog_EM.values()), 'g', label="Homogeneity")
plt.legend()
plt.xlabel("Number of cluster")
plt.ylabel("Silhouette Coefficient")

In [ ]:
gmm = GaussianMixture(n_components=2, max_iter=100, random_state=44, 
                      n_init=5).fit(bankX)
label = gmm.predict(bankX)
accuracy_score((1-label), bankY)

In [ ]:
kmeans = KMeans(n_clusters=31, n_init=50, max_iter=1000, random_state=44, 
                n_jobs=-2).fit(bankX)
label = kmeans.labels_

In [ ]:
X_kmeans = label.reshape(6819,1)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_kmeans, 
                                                    bankY, 
                                                    test_size=0.20, 
                                                    random_state=88,
                                                    stratify=bankY)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
def stratFold(train_X_all, train_y_all, classifier, metric, average=None,
              fold=5):
    train_metric, val_metric, indices = [], [], []
    for m in range(100, len(train_X_all), 100):
        train_X = train_X_all[:m]
        train_y = train_y_all[:m]
        skfolds = StratifiedKFold(n_splits=fold)
        metric_list = []
        metric_list_train = []
        for train_index, test_index in skfolds.split(train_X, train_y):
            train_X_folds = train_X[train_index, :]
            train_y_folds = train_y[train_index]
            test_X_fold = train_X[test_index, :]
            test_y_fold = train_y[test_index]
            classifier.fit(train_X_folds, train_y_folds)
            train_y_folds_pred = classifier.predict(train_X_folds)
            test_y_fold_pred = classifier.predict(test_X_fold)
            if average:
                metric_list.append(metric(test_y_fold, test_y_fold_pred,
                                     average=average))
                metric_list_train.append(metric(train_y_folds, train_y_folds_pred, 
                                            average=average))
            else:
                metric_list.append(metric(test_y_fold, test_y_fold_pred))
                metric_list_train.append(metric(train_y_folds, train_y_folds_pred))
                
        test_avg = sum(metric_list)/len(metric_list)
        train_avg = sum(metric_list_train)/len(metric_list_train)
        val_metric.append(test_avg)
        train_metric.append(train_avg)
        indices.append(m)
    
    return train_metric, val_metric, indices

In [ ]:
def plot_complexity(train_scores, val_scores, indices, title, inverse_x=True):
    plt.plot(indices, train_scores , "r-+", linewidth=2, label="train")
    plt.plot(indices, val_scores, "b-+", linewidth=2, label="validation")
    plt.title(title)
    plt.legend()
    plt.grid(color='gray', linestyle='-', linewidth=0.5, alpha=0.3)
    if inverse_x:
        plt.gca().invert_xaxis()

In [ ]:
def plot_learning_curve(train_scores, val_scores, indices, title, inverse_x=False):
    plt.plot(indices, train_scores , "r-+", linewidth=2, label="train")
    plt.plot(indices, val_scores, "b-+", linewidth=2, label="validation")
    plt.title(title)
    plt.legend()
    plt.grid(color='gray', linestyle='-', linewidth=0.5, alpha=0.3)
    if inverse_x:
        plt.gca().invert_xaxis()

In [ ]:
%%time

from sklearn.metrics import f1_score
f1_scores = stratFold(X_train, y_train, neural_net, f1_score, 
                      average="weighted")
plot_learning_curve(f1_scores[0], f1_scores[1], f1_scores[2], 
                    title="f1_weighted score learning curve")

In [ ]:
best_net = MLPClassifier(activation='relu', alpha=0.1, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=75, learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [ ]:
best_net.fit(X_kmeans, bankY)
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report
bank_train_predictions = best_net.predict(X_kmeans)
accuracy = accuracy_score(bankY, bank_train_predictions)
print(accuracy)
print(classification_report(bankY, bank_train_predictions))

In [ ]:
%%time

from sklearn.metrics import confusion_matrix
best_net.fit(X_train, y_train)
test_predictions = best_net.predict(X_test)
print(classification_report(y_test, test_predictions, digits=5))
print(confusion_matrix(y_test, test_predictions))

In [ ]:
%%time
gmm = GaussianMixture(n_components=31, max_iter=100, random_state=44, 
                      n_init=5).fit(bankX)
label = gmm.predict(bankX)

In [ ]:
X_em = label.reshape(6819,1)

In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.neural_network import MLPClassifier
neural_net = MLPClassifier()
neural_net.fit(X_em, bankY)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_em, 
                                                    bankY, 
                                                    test_size=0.33, 
                                                    random_state=88,
                                                    stratify=bankY)

In [ ]:
%%time

from sklearn.metrics import f1_score
f1_scores = stratFold(X_train, y_train, best_net, f1_score, 
                      average="weighted")
plot_learning_curve(f1_scores[0], f1_scores[1], f1_scores[2], 
                    title="f1_weighted score learning curve")

In [ ]:
HR = pd.read_csv("HR_data.csv")
HR_orig = pd.read_csv("HR_data.csv")

from sklearn.preprocessing import LabelEncoder

class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

HR['city']=HR['city'].astype(str)
HR['gender']=HR['gender'].astype(str)
HR['relevent_experience']=HR['relevent_experience'].astype(str)
HR['enrolled_university']=HR['enrolled_university'].astype(str)
HR['education_level']=HR['education_level'].astype(str)
HR['major_discipline']=HR['major_discipline'].astype(str)
HR['experience']=HR['experience'].astype(str)
HR['company_size']=HR['company_size'].astype(str)
HR['company_type']=HR['company_type'].astype(str)
HR['last_new_job']=HR['last_new_job'].astype(str)
HR = MultiColumnLabelEncoder(columns = ['city','gender','relevent_experience','enrolled_university','education_level','major_discipline','experience','company_size','company_type','last_new_job']).fit_transform(HR)


In [ ]:
hrX = HR.drop("target", 1).copy().values
hrY = HR["target"].copy().values
hr_x_df = HR.drop("target",1)
min_max_scaler = preprocessing.MinMaxScaler()
hrX = min_max_scaler.fit_transform(hrX)
hrX_df = pd.DataFrame(hrX, columns=hr_x_df.columns)

In [ ]:
clusters = list(range(1,150,1))

In [ ]:
%%time
sse = {}
for cluster in list(range(1,150,1)):
    kmeans = KMeans(n_clusters=cluster, max_iter=1000, random_state=44, n_jobs=-2).fit(hrX)
    sse[cluster] = kmeans.inertia_
plt.figure()
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Values of K")
plt.ylabel("Squared Error")
plt.show()

In [ ]:
%%time

from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

silh = {}
homog = {}
for cluster in range(3, 20, 1):
    kmeans = KMeans(n_clusters=cluster, n_init=50, max_iter=1000, random_state=44, 
                    n_jobs=-2).fit(hrX)
    label = kmeans.labels_
    sil_coeff = silhouette_score(hrX, label, metric='euclidean')
    homog_score = homogeneity_score(hrY, label)
    silh[cluster] = sil_coeff
    homog[cluster] = homog_score
    print("For n_clusters={}, The Silhouette Coefficient is {}".format(cluster, sil_coeff))
    print("For n_clusters={}, The homogeneity_score is {}".format(cluster, homog_score))
plt.figure()
plt.plot(list(silh.keys()), list(silh.values()))
plt.xlabel("Number of cluster")
plt.ylabel("Silhouette Coefficient")
plt.show()

In [ ]:
plt.figure()
plt.plot(list(silh.keys()), list(silh.values()), 'r', label="Silhouette")
plt.plot(list(homog.keys()), list(homog.values()), 'g', label="Homogeneity")
plt.legend()
plt.xlabel("Number of cluster")
plt.ylabel("Silhouette Coefficient")

In [ ]:
kmeans = KMeans(n_clusters=2, n_init=50, max_iter=1000, random_state=44, n_jobs=-2).fit(hrX)
accuracy_score((1-kmeans.labels_), hrY)

In [ ]:
%%time
silh_EM = {}
homog_EM = {}
log_likelihood_EM = {}
for cluster in range(2, 21, 1):
    gmm = GaussianMixture(n_components=cluster, max_iter=100, random_state=44, 
                          n_init=5).fit(hrX)
    label = gmm.predict(hrX)
    sil_coeff = silhouette_score(hrX, label, metric='euclidean')
    homog_score = homogeneity_score(hrY, label)
    silh_EM[cluster] = sil_coeff
    homog_EM[cluster] = homog_score
    log_likelihood_EM[cluster] = gmm.score(hrX)
    print("For n_clusters={}, The Silhouette Coefficient is {}".format(cluster, sil_coeff))
    print("For n_clusters={}, The homogeneity_score is {}".format(cluster, homog_score))
    print("For n_clusters={}, The log_likelihood score is {}".format(cluster, log_likelihood_EM[cluster]))

    
plt.figure()
plt.plot(list(silh_EM.keys()), list(silh_EM.values()))
plt.xlabel("Number of cluster")
plt.ylabel("Silhouette Coefficient")
plt.show()

In [ ]:
plt.figure()
plt.plot(list(silh_EM.keys()), list(silh_EM.values()), 'r', label="Silhouette")
plt.plot(list(homog_EM.keys()), list(homog_EM.values()), 'g', label="Homogeneity")
plt.legend()
plt.xlabel("Number of cluster")
plt.ylabel("Silhouette Coefficient")

In [ ]:
gmm = GaussianMixture(n_components=2, max_iter=100, random_state=44, 
                      n_init=5).fit(hrX)
label = gmm.predict(hrX)
accuracy_score((1-label), hrY)

In [ ]:
kmeans = KMeans(n_clusters=60, n_init=50, max_iter=1000, random_state=44, 
                n_jobs=-2).fit(hrX)
label = kmeans.labels_

In [ ]:
X_kmeans = label.reshape(19158,1)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_kmeans, 
                                                    hrY, 
                                                    test_size=0.25, 
                                                    random_state=42,
                                                    stratify=hrY)

In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.neural_network import MLPClassifier
neural_net = MLPClassifier()
neural_net.fit(X_kmeans, hrY)

In [ ]:
%%time
from sklearn.metrics import f1_score
f1_scores = stratFold(X_train, y_train, neural_net, f1_score, 
                      average="macro")
plot_learning_curve(f1_scores[0], f1_scores[1], f1_scores[2], 
                    title="f1_weighted score learning curve")